In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from typing import Any, Callable, NamedTuple, Tuple, Union
Step = int
Schedule = Callable[[Step], float]

from IPython import display
from IPython.display import Image, clear_output
from PIL import Image
import glob, os, shutil
import os.path

import time

import scipy.io as io
import scipy.sparse.csgraph as csgraph
from scipy.sparse.csgraph import laplacian as csgraph_laplacian
import scipy as sp
from scipy.stats import gaussian_kde
from scipy.linalg import null_space

import jax
from jax import jit, vmap, random, grad, value_and_grad, hessian
from jax.experimental import optimizers, sparse
from jax.experimental.optimizers import optimizer
from jax import numpy as jnp

from functools import partial
import itertools

import math
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import collections as mc

import seaborn as sns

import datetime
from tqdm.notebook import tqdm

import networkx as nx

%load_ext autoreload
%autoreload 2

In [3]:
import utils
from utils import *
from optimizers import *

In [4]:
# load the data from the SuiteSparse Matrix Collection format
# https://www.cise.ufl.edu/research/sparse/matrices/
graphs = ['qh882','dwt_1005','3elt','commanche_dual','bcsstk31']
graphdir = './testcases/'
graphpostfix = '3elt'
assert graphpostfix in graphs
grid_testcase = nx.grid_graph(dim=(50, 50))
#grid_testcase = nx.triangular_lattice_graph(25,25)
#grid_testcase = nx.cycle_graph(100)
grid_testcase_adjacency = nx.adjacency_matrix(grid_testcase).toarray().astype(np.int16)
DEBUG=True
if DEBUG:
    graph, G, A, L, D, n = load_graph(graphdir+graphpostfix, A=grid_testcase_adjacency, plot_adjacency=False, verbose=True)
else:
    graph, G, A, L, D, n = load_graph(graphdir+graphpostfix, A=None, plot_adjacency=False, verbose=True)


Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 4900
Average degree:   3.9200


In [5]:
del G
del A
del D

In [6]:
@jit
def project(X1, C, E_0, c=jnp.array([0,0])):
    C1 = X1.T@X1
    C1sqrt = utils._sqrtm(C1)
    Csqrt = utils._sqrtm(C)
    U,s,V = jnp.linalg.svd(Csqrt@C1sqrt)
    X = X1@jnp.linalg.inv(C1sqrt)@U@V.T@Csqrt

    U_E, _, V_E = jnp.linalg.svd(X.T@E_0)
    X = X@(-U_E@V_E.T)
    return X.real

@partial(jit, static_argnums=(3,))
def step(i, opt_state, Z, opt_update):
    """Perform a single descent + projection step with arbitrary descent direction."""
    return opt_update(i, Z, opt_state)

def _D_Z(X, A, P, d, e):
    I = jnp.eye(A.shape[0])
    Ad = A + d*I
    
    Del = jnp.linalg.solve(X.T@jnp.linalg.solve(Ad, X), X.T)@jnp.linalg.solve(Ad, e)
    Z = jnp.linalg.solve(Ad, -X@Del + e)
    
    return Del, Z

@jit
def _sqp(A, P, L, E_0, X):
    """Perform an iteration of SQP.""" 
    w = jnp.linalg.eigvals(L)
    idx = w.argsort() 
    w = w[idx]
    E = -E_0 - (A@X + X@L)

    Del_0, Z_0 = _D_Z(X, A, P, w[0], E[:,0])
    Del_1, Z_1 = _D_Z(X, A, P, w[1], E[:,1])
    
    Z = jnp.stack([Z_0, Z_1], axis=1)
    Del = jnp.stack([Del_0, Del_1], axis=1)
    
    return Z, Del  
@jit
def D_Z(X, A, P, d, e, I):
    Ad = A + d*I

    #ADinvP = jnp.linalg.solve(Ad, P.T)
    sp_solve = lambda A, b:jnp.array(sp.linalg.solve(A,b))
    #sp_solve = lambda _, x: jax.scipy.sparse.linalg.bicgstab(lambda b:Ad@b, x, M=lambda b:M@b, maxiter=100)[0]
    #sp_solve = lambda _, x: jax.scipy.sparse.linalg.gmres(lambda b:Ad@b , x, maxiter=100, solve_method='incremental',M=jnp.linalg.inv(Ad))[0]
    ADinvP = sp_solve(Ad, P.T)
    
    #Del = jnp.linalg.solve(X.T@(jnp.linalg.solve(Ad,X)),X.T)@jnp.linalg.solve(Ad,e)
    Del = jnp.linalg.solve(X.T@(sp_solve(Ad,X)),X.T)@sp_solve(Ad,e)
    
    Z = P@(ADinvP@(P@(-X@Del + e)))

    return Del, Z

@jit
def sqp(X, A, P, L, E_0, I):
    """Perform an iteration of SQP.""" 
    w = jnp.linalg.eigvals(L)
    idx = w.argsort()  
    w = w[idx].real
    E = -E_0 - (A@X + X@L)
    Del_0, Z_0 = D_Z(X, A, M, P, w[0], E[:,0], I)
    Del_1, Z_1 = D_Z(X, A, M, P, w[1], E[:,1], I)
    
    Z = jnp.stack([Z_0, Z_1], axis=1)
    Del = jnp.stack([Del_0, Del_1], axis=1)
    
    return Z, Del  

def scipy_sqp(X, A, P, L, E_0, I):
    """Perform an iteration of SQP.""" 
    w = jnp.linalg.eigvals(L)
    idx = w.argsort()  
    w = w[idx].real
    E = -E_0 - (A@X + X@L)
    Del_0, Z_0 = scipy_D_Z(X, A, P, w[0], E[:,0], I)
    Del_1, Z_1 = scipy_D_Z(X, A, P, w[1], E[:,1], I)
    
    Z = jnp.stack([Z_0, Z_1], axis=1)
    Del = jnp.stack([Del_0, Del_1], axis=1)
    
    return Z, Del  

def scipy_D_Z(X, A, P, d, e, I):
    Ad = A + d*I

    #ADinvP = jnp.linalg.solve(Ad, P.T)
    #sp_solve = lambda _, x: bicgstab(lambda b:Ad@b , x, maxiter=100)[0]
    Ad = sp.sparse.coo_matrix((A.data, (A.indices[:,0], A.indices[:,1]))).tocsr()
    sp_solve = lambda A, b:jnp.array(sp.sparse.linalg.spsolve(A,b))
    #sp_solve = lambda A, b:jnp.array(sp.sparse.linalg.bicgstab(A,b))
    ADinvP = sp_solve(Ad, P.T)
    
    Del = jnp.linalg.solve(X.T@(sp_solve(Ad,X)),X.T)@sp_solve(Ad,e)
    
    Z = P@(ADinvP@(P@(-X@Del + e)))

    return Del, Z


def newton(opt_params, A, P, L, C, X_k, b_x, b_y, convergence_criterion, 
           maxiters=20, alpha=1e-2, beta=0.9, initL=True):
    """Perform iterations of PND + backtracking line search."""    
    opt_state, opt_update, get_params = opt_params
    X_k = get_params(opt_state)
    E_0 = np.stack([b_x, b_y], axis=1)
   
    pAp = P@A@P.T

    if initL:
        L = L_init(X_k, C, pAp, E_0)
    
    report = {'x':None, 'lossh':[f(X_k, pAp, pAp, b_x, b_y).item()], 'sln_path':[np.asarray(X_k)], 
            'foc':[foc_sqp(X_k, L, C, pAp, E_0).item()], 'step_sizes':[1], 'L':[L]}
    
    cc = 0
    
    for k in tqdm(range(maxiters)):         
        #Z, Del = sqp(A, P, pAp, L, E_0, X_k)
        Z, Del = _sqp(pAp, P, L, E_0, X_k)
        
        # backtracking line search
        f_xp = 1e8
        stp = 1
        f_x, gr = value_and_grad(f)(X_k, pAp, pAp, b_x, b_y)
        len_p = jnp.linalg.norm(Z)
        X_k_t = X_k
        
        opt_state_t = opt_state
        
        while f_xp >= f_x:
            stp *= beta
            opt_state_t = step(stp, opt_state, -Z, opt_update)
            X_k_t = get_params(opt_state_t)
            f_xp = f(X_k_t, pAp, pAp, b_x, b_y)
        
            if stp * len_p < 1e-8:
                break  
                
        L = L + stp*Del
        foc = foc_sqp(X_k, L, C, pAp, E_0)
        
        opt_state = opt_state_t
        X_k = get_params(opt_state_t)
        
        if f_xp > report['lossh'][-1]:
            break
        
        report['sln_path'].append(np.asarray(X_k))
        report['step_sizes'].append(stp)
        report['foc'].append(foc.item())
        report['lossh'].append(f_xp.item())
        report['L'].append(np.asarray(L))
        
        if len(report['lossh']) > 2 and np.abs(foc.item()) <= convergence_criterion:
            cc += 1
            if cc > 10:
                print('converged')
                break
        if cc > 0:
            cc -= 1
            
    return report

@jit
def subspace(X_k_q, X_k, Z, v, A, E_0, E_00, P, C):
    AXE = P@(A@X_k+E_0)
    Q, _ = jnp.linalg.qr(jnp.concatenate([X_k_q, Z, v, AXE],axis=-1), mode='reduced')
    #Q, _ = jnp.linalg.qr(jnp.concatenate([X_k_q, v, AXE],axis=-1), mode='reduced')
    PQ = P.T@Q
    B=PQ.T@(A@PQ)
    
    #X_k = jnp.linalg.pinv(PQ)@X_k
    X_k = PQ.T@X_k
    E_0 = PQ.T@E_00
    X_k = project(X_k, C, E_0)
    
    w_v, v_v = jnp.linalg.eig(B)
    idx = w_v.argsort()
    v_v = v_v[idx]
    v = Q@v_v[:,1:3]
    PQE_0 = PQ@E_0
    
    return Q, PQ, B, X_k, E_0, PQE_0, v
    
def ssm(opt_params, A, P, L, C, X_k, b_x, b_y, convergence_criterion, 
        maxiters=10, alpha=1e-2, beta=0.9):
    """
    1. compute newton direction z = sqp(X, Z, v, Ax + E0) & subspace S
    2. approximate locally optimal X, L on S; X = min F(\hat{X}, B, V.T@E0)
    """
    opt_state, opt_init, opt_update, get_params = opt_params
    X_k = get_params(opt_state)
    
    E_00 = jnp.stack([b_x, b_y], axis=1)
       
    cc = 0
    L = jnp.eye(2)
    results = None
    E_0 = E_00
    #X_k_q = P@X_k
    X_k_q = X_k
    #v = jnp.zeros_like(X_k_q)
    
    w_v, v_v = jnp.linalg.eig(P@A@P.T)
    idx = w_v.argsort()
    v_v = v_v[idx]
    v = v_v[:,1:3]
    
    I = jnp.eye(A.shape[0])
    I = sparse.BCOO.fromdense(I)
    M = None 
    for k in tqdm(range(maxiters)):     
        'Subspace computation'
        #Z, Del = sqp(X_k, A, Ad, M, P, L, E_0, I)
        Z, Del = scipy_sqp(X_k, A, P, L, E_0, I)
        #Z = None

        'initialize wrt subspace'
        #if k > 1:
        #    print('before sub',f(X_k, A, A, E_00[:,0], E_00[:,1]))
        qq, Q, B, X_k, E_0, QE_0, _ = subspace(X_k_q, X_k, Z, v, A, E_0, E_00, P, C)
        #print('after sub',f(Q@X_k, A, A, E_00[:,0], E_00[:,1]))
        
        opt_init, opt_update, get_params = psgd(partial(lambda x, y, z: z, 
                                                        E_0, C))    
        
        opt_state = opt_init(X_k)       
        result = newton((opt_state, opt_update, get_params), A, Q.T, L, C, X_k, E_0[:,0], E_0[:,1], 
                        convergence_criterion=convergence_criterion, maxiters=20, alpha=0.0, beta=0.9, initL=True) 
        
        X_k = result['sln_path'][-1]
        L = result['L'][-1]
        #print('inner',f(X_k, Q.T@A@Q, Q.T@A@Q, E_00[:,0]@Q, E_00[:,1]@Q))
        X_k_q = qq@X_k # QX
        X_k = Q@X_k    # PQX
        E_0 = QE_0     # PQE
        X_k = project(X_k, C, E_0)
        #print('outer',f(X_k, A, A, E_0[:,0], E_0[:,1]))
        if results == None:
            results = result
            results['sln_path'] = [X_k]
            results['lossh'] = [result['lossh'][-1]]            
        
        results['lossh'].extend(result['lossh'])
        results['sln_path'].extend([X_k]*len(result['lossh']))
        results['foc'].extend(result['foc'])
        results['step_sizes'].extend(result['step_sizes'])
    return results

In [7]:
def transform_A(A, X_k, boolean_idx):
    boolean_fixed_idx, boolean_nonfixed_idx = boolean_idx
    X_1x = X_k[boolean_fixed_idx,0]
    X_1y = X_k[boolean_fixed_idx,1]
    X_2  = X_k[boolean_nonfixed_idx]
    A_12 = A[boolean_fixed_idx, :]
    A_12 = A_12[:, boolean_nonfixed_idx] 
    b_x = X_1x@A_12
    b_y = X_1y@A_12
    A = A[boolean_nonfixed_idx, :]
    A = A[:,boolean_nonfixed_idx]
    
    return A, X_1x, X_1y, X_2, b_x, b_y

def map_vars(A, X_k, fixed_idx, centercons, decomp=True):
    """Preprocess variables """ 
    N = A.shape[0] 
    k = fixed_indices.shape[0]
    fixed_idx = jnp.zeros((k,N))
    for i in range(k):
        fixed_idx=jax.ops.index_add(fixed_idx,jnp.index_exp[i, fixed_indices[i]],1)

    boolean_fixed_idx = fixed_idx.sum(0).astype(bool)
    boolean_nonfixed_idx = (1-fixed_idx.sum(0)).astype(bool)
    
    A, X_1x, X_1y, X_2, b_x, b_y = transform_A(A, X_k, (boolean_fixed_idx, boolean_nonfixed_idx))
    X_k = X_2
    
    P = jnp.eye(A.shape[0])

    constraints = np.expand_dims(np.ones(X_2.shape[0]),0)
    #P = qr_null(constraints).T
    Q, rnk = qr_null(constraints)
    Q = Q[:, rnk:].conj().T
    P = Q * np.sign(P[0,0])

    #pinvcons = jnp.linalg.pinv(constraints)
    #n0_x = pinvcons@(np.expand_dims(centercons[0],0))
    n0_x = jnp.zeros_like(b_x)
    if centercons[1] == centercons[0]:
        n0_y = n0_x
    else:
        n0_y = pinvcons@(np.expand_dims(centercons[1],0))

    return X_k, A, P, b_x, b_y, n0_x, n0_y, fixed_idx
    

def cluster(rng, opt_params, X_k, fixed_x, A, mapped_vars, fixed_indices=None, maxiters=1000, convergence_criterion=1e-3,
            c1=1, c2=1, c3=0, centroid=jnp.array([0,0]), centercons=None, v=None, D=None, eps=1e-8, method='pgd'):
    """Given an adjacency matrix A and initialization X_k, optimize X."""
    method = method.lower()
    opt_init, opt_update, get_params = opt_params    
        
    k = fixed_x.shape[0]
    if fixed_indices is None:
        fixed_coordsx = fixed_x[:,0]
        fixed_coordsy = fixed_x[:,1]
    else:
        fixed_coordsx = X_k[fixed_indices,0]
        fixed_coordsy = X_k[fixed_indices,1]
    
    N = A.shape[0]

    if v is None:
        v = jnp.ones(N)
    if D is None:
        D = jnp.diag(v)
    if centercons is None:
        centercons = jnp.zeros(2)
    A, P, b_x, b_y, n0_x, n0_y, fixed_idx = mapped_vars

    C = jnp.block([[c1, c3],[c3, c2]])
    assert jnp.linalg.det(C) > 1e-5
    
    E_0 = jnp.stack([b_x, b_y], axis=1)
    n0 = jnp.stack([n0_x,n0_y],axis=0)
    #X_k_n = jnp.array(np.linalg.pinv(P.T)@(X_k-n0.T))
    
    ######
    #X_k_n = X_k
    ######
    w_v, v_v = jnp.linalg.eig(P@A@P.T)
    idx = w_v.argsort()
    v_v = v_v[idx]
    X_k_n = v_v[:,1:3]
    
    
    #X_k_n = project(X_k_n, C, E_0, centercons)
    L = jnp.eye(2)

    opt_state = opt_init(X_k_n) #pap
    result = ssm((opt_state, opt_init, opt_update, get_params), A, P, L, C, X_k_n, b_x, b_y, 
                    convergence_criterion=convergence_criterion, maxiters=maxiters, alpha=0.0, beta=0.9) 

    X_k = result['sln_path'][np.argmin(result['lossh'])]

    X_k_n = np.zeros((N,2))
    if fixed_indices is None:
        X_k_n[:,0] = np.concatenate([fixed_coordsx, np.array(X_k[:,0]) + n0_x.T])
        X_k_n[:,1] = np.concatenate([fixed_coordsy, np.array(X_k[:,1]) + n0_y.T])
    else:
        X_k_n[:,0] = np.array(P.T@X_k[:,0]) + n0_x.T
        X_k_n[:,1] = np.array(P.T@X_k[:,1]) + n0_y.T
        
    result['x'] = X_k_n
    
    mask = (1-fixed_idx.sum(0)).astype(np.bool)
    result['mask'] = mask
    result['centroid'] = centercons
    if fixed_idx.sum() == 0:
        result['g'] = np.array(g(X_k_n, v, centercons))
        result['h'] = np.array(h(X_k_n, np.diag(v), c1, c2, c3, centroid))      
    else:
        # need to insert at mask, not cat/stack
        result['g'] = np.array(g(X_k_n[mask], v[mask], centercons))
        result['h'] = np.array(h(X_k_n[mask], np.diag(v[mask]), c1, c2, c3, centroid))
    result['P'] = (P)
    result['e'] = np.vstack([b_x,b_y])
    result['n'] = (n0_x, n0_y)

    return result


In [8]:
##### USER PARAMETERS #####
method = "ssm" # pnd, ssm, or pgd


seed = 0 # random seed
eps = 1e-8 # global epsilon variable
rng = random.PRNGKey(seed)
key, subkey = jax.random.split(rng)

v = np.ones(n)
c1=v.sum()*10**2*1/12
c2=v.sum()*10**2*1/12
c3=0
C = jnp.block([[c1, c3],[c3, c2]])

X_k_r = (random.normal(subkey, (n,2))*np.sqrt(10))

if os.path.isfile(graphdir+graphpostfix+'_evals.npy') and \
   os.path.isfile(graphdir+graphpostfix+'_evecs.npy'):
    w = np.load(graphdir+graphpostfix+'_evals.npy')
    v = np.load(graphdir+graphpostfix+'_evecs.npy')    
else:
    w,v = sp.sparse.linalg.eigsh(L, k=min(n,5), which='SM')
    np.save(graphdir+graphpostfix+'_evals.npy',w)
    np.save(graphdir+graphpostfix+'_evecs.npy',v)
if DEBUG:
    w,v = sp.sparse.linalg.eigsh(L, k=min(n,5), which='SM')
X_k = v[:,1:3].real

if DEBUG:
    fixed_indices = np.array([0])
else:
    fixed_indices = np.array([0,1,2,3,4,5,6,7,8,9,10])


X_k[fixed_indices] = X_k_r[fixed_indices]

In [9]:
del w
del v
#del X_k_r
v = jnp.ones(n)

X_k, A, P, b_x, b_y, n0_x, n0_y, fixed_idx = map_vars(L, X_k, fixed_indices, v.sum()*jnp.array([0,0]))
A = sparse.BCOO.from_scipy_sparse(A)
mapped_vars = (A, P, b_x, b_y, n0_x, n0_y, fixed_idx)

if method == "pgd":
    pgd_lr = 5e-2
    opt_init, opt_update, get_params = padam(pgd_lr,partial(lambda x, y, z: project(z, y, x), 
                                                    np.stack([b_x,b_y],axis=1), C), b1=0.9, b2=0.999, eps=1e-08)
elif method == "pnd":
    opt_init, opt_update, get_params = psgd(partial(lambda x, y, z: project(z, y, x), 
                                                    np.stack([b_x,b_y],axis=1), C))
elif method == 'ssm':
    opt_init, opt_update, get_params = psgd(partial(lambda x, y, z: project(z, y, x), 
                                                    np.zeros((8,2)), C))       
else:
    print('method not supported')
    assert False

In [10]:
#%%prun

result = cluster(rng, (opt_init, opt_update, get_params), 
                 X_k, X_k_r[fixed_indices], L, mapped_vars, fixed_indices=None, c1=c1, c2=c2, c3=c3, centercons=v.sum()*jnp.array([0,0]), 
                 v=None, D=None, eps=1e-8, maxiters=10, convergence_criterion=1e-3, method=method)
results = [result]
X_k_n=result['x']

  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: dot_general requires contracting dimensions to have the same shape, got [2499] and [2498].

In [ ]:
resgraph = utils.plot_results(result)

In [ ]:
voxel_id, voxel_bound = voxel_cluster(X_k, np.array([5, 5]))

In [ ]:
#utils.plot_graph(X_k, graph, title='loss: {} h: {} g: {} foc: {}'.format(str(np.round(np.min(result['lossh']),2)), 
#                                                                            np.round(result['g'],2), np.round(result['g'],2), 
#                                                                           str(np.round(result['foc'][np.argmin(result['lossh'])],2))), fixed_indices=fixed_indices, c=voxel_id)

In [ ]:
utils.plot_graph(X_k_n, graph, title='loss: {} h: {} g: {} foc: {}'.format(str(np.round(np.min(result['lossh']),2)), 
                                                                            np.round(result['h'],2), np.round(result['g'],2), 
                                                                           str(np.round(result['foc'][np.argmin(result['lossh'])],2))), fixed_indices=fixed_indices, c=None)

In [ ]:
#utils.plot_animation(results, graph, fixed_coordinates=X_k_r[fixed_indices])